In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
# import datetime

In [2]:
data = pd.read_csv('feature_matrix.csv').drop(['Unnamed: 0'], axis=1)

In [3]:
data.head()

,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,...,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points
0,8448208,20,2015,H,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
1,8448208,35,2015,A,2.0,NaN,NaN,NaN,2.000000,0.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,3.0
2,8448208,41,2015,A,0.0,NaN,NaN,NaN,1.000000,0.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,8.0
3,8448208,55,2015,H,0.0,NaN,NaN,NaN,0.666667,1.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,29.5
4,8448208,68,2015,H,2.0,NaN,NaN,NaN,1.000000,1.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,9.3


In [4]:
data.describe()

,player_id,game_id,season,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,assists_5,...,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points
count,2.175000e+05,217500.000000,217500.000000,216053.000000,210636.000000,204354.000000,192640.000000,213317.000000,216053.000000,210636.000000,...,210636.000000,204354.000000,192640.000000,213317.000000,216053.000000,210636.000000,204354.000000,192640.000000,213317.000000,217500.000000
mean,8.475073e+06,606.442828,2016.957641,0.159040,0.160458,0.161908,0.164392,0.160805,0.266786,0.268982,...,0.004699,0.004759,0.004882,0.004889,0.004777,0.004833,0.004900,0.005025,0.004807,6.745767
std,3.163786e+03,351.712966,1.393767,0.413574,0.211082,0.169211,0.143517,0.157923,0.536956,0.278307,...,0.031226,0.022782,0.016986,0.020905,0.069350,0.031542,0.022749,0.016723,0.019313,6.507388
min,8.448208e+06,1.000000,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.473573e+06,303.000000,2016.000000,0.000000,0.000000,0.000000,0.050000,0.048780,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000
50%,8.475753e+06,605.000000,2017.000000,0.000000,0.000000,0.100000,0.150000,0.125000,0.000000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.500000
75%,8.477337e+06,907.000000,2018.000000,0.000000,0.200000,0.200000,0.250000,0.233333,0.000000,0.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
max,8.481813e+06,1270.000000,2019.000000,5.000000,2.200000,1.600000,1.050000,4.000000,5.000000,2.400000,...,0.600000,0.400000,0.250000,1.000000,2.000000,0.600000,0.300000,0.200000,1.000000,64.800000


In [5]:
## Replace NAN with Mean 
data = data.fillna(data.mean())
data = data.replace('H', 1)
data = data.replace('A', 0)

In [6]:
### drop columns that are not necessary

data = data.drop(['player_id', 'game_id', 'season'], axis = 1)

In [7]:
data

,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,assists_5,assists_10,assists_20,...,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points
0,1,0.15904,0.160458,0.161908,0.164392,0.160805,0.266786,0.268982,0.271206,0.275341,...,0.004699,0.004759,0.004882,0.004889,0.004777,0.004833,0.0049,0.005025,0.004807,21.5
1,0,2.00000,0.160458,0.161908,0.164392,2.000000,0.000000,0.268982,0.271206,0.275341,...,0.004699,0.004759,0.004882,0.000000,0.000000,0.004833,0.0049,0.005025,0.000000,3.0
2,0,0.00000,0.160458,0.161908,0.164392,1.000000,0.000000,0.268982,0.271206,0.275341,...,0.004699,0.004759,0.004882,0.000000,0.000000,0.004833,0.0049,0.005025,0.000000,8.0
3,1,0.00000,0.160458,0.161908,0.164392,0.666667,1.000000,0.268982,0.271206,0.275341,...,0.004699,0.004759,0.004882,0.000000,0.000000,0.004833,0.0049,0.005025,0.000000,29.5
4,1,2.00000,0.160458,0.161908,0.164392,1.000000,1.000000,0.268982,0.271206,0.275341,...,0.004699,0.004759,0.004882,0.000000,0.000000,0.004833,0.0049,0.005025,0.000000,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217495,0,0.00000,0.000000,0.000000,0.100000,0.106430,0.000000,0.000000,0.000000,0.050000,...,0.000000,0.000000,0.000000,0.002217,0.000000,0.000000,0.0000,0.000000,0.002217,1.5
217496,0,0.00000,0.000000,0.000000,0.100000,0.106195,0.000000,0.000000,0.000000,0.050000,...,0.000000,0.000000,0.000000,0.002212,0.000000,0.000000,0.0000,0.000000,0.002212,1.5
217497,1,0.00000,0.000000,0.000000,0.100000,0.105960,0.000000,0.000000,0.000000,0.050000,...,0.000000,0.000000,0.000000,0.002208,0.000000,0.000000,0.0000,0.000000,0.002208,1.5
217498,0,0.00000,0.000000,0.000000,0.050000,0.105727,0.000000,0.000000,0.000000,0.050000,...,0.000000,0.000000,0.000000,0.002203,0.000000,0.000000,0.0000,0.000000,0.002203,1.5


In [8]:
## Replace NAN with Mean 
y = data[data.columns[-1]]
X = data[data.columns[:-1]]


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [10]:
train_data = lgb.Dataset(X_train, y_train)

In [12]:
param = {'num_leaves':150,'objective': 'regression','max_depth':7,'learning_rate':.05,'max_bin':200}

In [13]:
num_round=50
lgbm=lgb.train(param,train_data,num_round)


In [14]:
pred = lgbm.predict(X_test)

In [15]:
pred

array([5.3242507 , 5.17588846, 5.84393019, ..., 8.37237619, 8.63372216,
       7.5820081 ])

In [16]:
mean_squared_error(y_test, pred)

35.52741091283387

In [17]:
lgbm.feature_importance

<bound method Booster.feature_importance of <lightgbm.basic.Booster object at 0x0000023DF7A55E48>>